In [1]:
import pandas as pd

import numpy as np

import xlsxwriter, os

import matplotlib.pyplot as plt

%matplotlib inline

from openpyxl import load_workbook

from sklearn import preprocessing

In [2]:
# save sales by industry into excel files

def create_excel_fnc(path, df, sheetname):
    
    writer = pd.ExcelWriter(path)
    
    df.to_excel(writer, sheetname)
    
    writer.save()
    
def add_sheet_fnc(path, df,sheetname):
    
    book = load_workbook(path)
    
    writer = pd.ExcelWriter(path, engine='openpyxl')
    
    writer.book = book
    
    df.to_excel(writer, sheet_name=sheetname)
    
    writer.save()
    
    writer.close()

# Process MN Industry 2015 Dataset

In [3]:
# load MN Industry Dataset

MN_df = pd.read_excel(os.getcwd()+r'\MN_Data\MN_Industry2015.xls')

In [4]:
# drop unexpected rows

MN_df = MN_df[MN_df.COUNTY != 'NON-MINNESOTA CO']

In [5]:
# extract all industry names

Industry_Names = MN_df.INDUSTRY.unique()

In [6]:
Industry_Names[33] = '999 UNDESIGNATED or SUPPRESSED'

In [7]:
MN_df.head()

,YEAR,COUNTY,INDUSTRY,GROSS SALES,TAXABLE SALES,SALES TAX,USE TAX,TOTAL TAX,NUMBER
0,2015,AITKIN,236 CONSTRUCT -BUILDINGS,1501102,172758,15100,1314,16414,4
1,2015,AITKIN,238 CONSTRUCT -SPECIAL TRADES,16217060,2286540,157199,22557,179756,31
2,2015,AITKIN,311 MFG -FOOD,3502899,54989,3781,1,3782,4
3,2015,AITKIN,314 MFG -TEXTILE PROD MILLS,207331,131150,9016,0,9016,5
4,2015,AITKIN,321 MFG -WOOD PRODUCT,28025899,203773,14009,1507,15516,6


In [8]:
# The count of the industries

# Industry_Names

In [15]:
Food_df = MN_df[MN_df.INDUSTRY == '311 MFG -FOOD']
    
Food_County = Food_df.COUNTY.tolist()

In [17]:
Food_County_Sales = Food_df[['COUNTY', 'GROSS SALES']]

In [18]:
Food_County_Sales.head()

,COUNTY,GROSS SALES
2,AITKIN,3502899
42,ANOKA,7361521
113,BECKER,750425
160,BELTRAMI,294666
277,BLUE EARTH,1383165880


## Save MN Sales data for each industry by county

In [9]:
County_Names = MN_df.COUNTY.unique().tolist()

In [11]:
for industry_name in Industry_Names:
    
    df = MN_df[MN_df.INDUSTRY == industry_name]
    
    Industry_County = df.COUNTY.tolist()      

    Industry_County_Sales = df[['COUNTY', 'GROSS SALES']]

    Industry_County_Sales.set_index('COUNTY', inplace=True)

    path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Sales_Data\Industry_Sales.xlsx'
    
#     add_sheet_fnc(path1, Industry_County_Sales,industry_name)

# Load our raw data by County

In [12]:
# load our data

Raw_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Raw_Data.xlsx')

Industry_Sales_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Sales_Data\Industry_Sales.xlsx')

# Process Indices

In [13]:
sheetnames = Raw_Data.sheet_names

sheetnames.remove('Nat_Frn_Popn')

indices_names = sheetnames[0:8]

In [20]:
for industry_name in Industry_Names:
    
#     writer = pd.ExcelWriter(industry_name+'.xlsx', engine='xlsxwriter')
    
    sales_df = MN_df[MN_df.INDUSTRY == industry_name]
    
    Industry_County = sales_df.COUNTY.tolist()
    
#     path_raw = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Indices_Industry'
        
#     writer_raw = pd.ExcelWriter(path_raw+'\\'+industry_name+'.xlsx', engine='xlsxwriter')
    
    path_corr = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Indices_Industry'
        
    writer_corr = pd.ExcelWriter(path_corr+'\\'+industry_name+'.xlsx', engine='xlsxwriter')
    
#     industry_sales_df = Industry_Sales_Data.parse(industry_name)

    industry_sales_df = sales_df[['COUNTY', 'GROSS SALES']]
    
    for sheetname in indices_names:
        
        raw_df = Raw_Data.parse(sheetname)
        
        raw_df.set_index('County', inplace=True)
    
        raw_df = raw_df.loc[Industry_County]
        
#         raw_df.to_excel(writer_raw, sheet_name=sheetname)

        corr_df = pd.DataFrame()       

        corr_df['Correlation Coefficient'] = raw_df[raw_df.columns[:]].apply(lambda x: x.corr(industry_sales_df['GROSS SALES']))
                
        corr_df.to_excel(writer_corr, sheet_name=sheetname)       

In [ ]:
# industry_sales_df = Industry_Sales_Data.parse('311 MFG -FOOD')

# industry_sales_df.head()

In [ ]:
# sales_df = MN_df[MN_df.INDUSTRY == '311 MFG -FOOD']
    
# sales_df.head()

In [ ]:
# Industry_County = sales_df.COUNTY.tolist()
# Industry_County

In [ ]:
# raw_df = Raw_Data.parse('JCI Monthly')
# raw_df.set_index('County', inplace=True)
# raw_df = raw_df.loc[Industry_County]
# raw_df.head()

In [ ]:
# raw_df = raw_df.astype('float64')
# industry_sales_df = industry_sales_df['GROSS SALES'].astype('float64')
# raw_df.head()

In [14]:
# corr_df = pd.DataFrame()

In [ ]:
# corr_df['Correlation Coefficient'] = raw_df[raw_df.columns[:]].apply(lambda x: x.corrwith(industry_sales_df['GROSS SALES']))

# corr_df

In [ ]:
def gen_indices_data(path_raw, path_corr, sheetname, raw_data):
    
    df = raw_data.parse(sheetname)
    
    df.set_index('County', inplace=True)
    
    df = df.loc[Food_County]
    
    add_sheet_fnc(path_raw, df, sheetname)
    
    corr_df = pd.DataFrame()

    corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(Food_County_Sales['GROSS SALES']))

    add_sheet_fnc(path_corr, corr_df, sheetname)

In [ ]:
sheetnames = Raw_Data.sheet_names

sheetnames.remove('Nat_Frn_Popn')

indices_names = sheetnames[0:8]

In [ ]:
path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Food\Indices.xlsx'

path2 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Food\Indices_Corr.xlsx'

for sheetname in indices_names:
    
    gen_indices_data(path1, path2, sheetname, Raw_Data)

# Process Category Dataset

In [ ]:
def create_Categories_Data(path_raw, path_corr, raw_data, sheetname):

    df = raw_data.parse(sheetname)
    
    variable_names = df.loc[0][1:]

    df.set_index('County', inplace=True)
    
    df = df.loc[Food_County]
    
    raw_df = df.reset_index()

    raw_df.loc[-1] = variable_names

    raw_df.index = raw_df.index+1

    raw_df = raw_df.sort_index()
    
    df = df.astype('float64')

    add_sheet_fnc(path_raw, raw_df,sheetname)
    
    corr_df = pd.DataFrame()
    
    corr_df['Variable Name'] = variable_names

    corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(Food_County_Sales['GROSS SALES']))

    add_sheet_fnc(path_corr, corr_df, sheetname)

In [ ]:
category_names = sheetnames[9:]

In [ ]:
path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Industry_Raw_Data\Food\Categories.xlsx'

path2 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Industry_Correlation_Data\Food\Categories_Corr.xlsx'

In [ ]:
for sheetname in category_names:
    
    create_Categories_Data(path1, path2, Raw_Data, sheetname)